In [94]:
pip install pandas plotly openpyxl nbformat dash rapidfuzz ace_tools


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [95]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [96]:
fichier_xls = "../data/bnp_export_15_03.xlsx"
df = pd.read_excel(fichier_xls)
df.head()

,id,Date operation,Categorie operation,Sous Categorie operation,Libelle operation,Montant operation,Date Extraction,Jour,Mois,Année,Mois-Annee,is_subscription
0,77f8493ca24f02dea17cb67965c5258a73fc4fde75e7b2...,2025-03-14,A Categoriser,Virement interne,VIREMENT INTERNE VERS MADEMOISELLE LAURY FYOT ...,-64878.00,2025-03-15 10:47:36,14,3,2025,03-2025,False
1,942d2e6eae9a231cc72985bdc08804aada0b9d36d372a7...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech","PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...",-11.99,2025-03-15 10:47:36,14,3,2025,03-2025,True
2,941bb1735f80c2267aac1c36206a2eae90058e2f740d9a...,2025-03-14,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB BISTRO DU FAUBO (FRANCE) DU 12/03/...,-7.00,2025-03-15 10:47:36,14,3,2025,03-2025,False
3,a09fdabc5afb7f14f4275c530543500f1e1294e929fd2f...,2025-03-14,Vie Quotidienne,"Achat multimedia, hightech",PAIEMENT CB APPLE (IRLANDE) DU 13/03/25 - CART...,-3.99,2025-03-15 10:47:36,14,3,2025,03-2025,True
4,4b45642f04eeb3b3f50599b08b28c403fe6cf684a80ae5...,2025-03-13,Loisirs et Sorties,"Restaurants, bars",PAIEMENT CB CARAVANA (FRANCE) DU 12/03/25 - CA...,-13.90,2025-03-15 10:47:36,13,3,2025,03-2025,False


In [97]:
df["Mois"] = df["Date operation"].dt.to_period("M")
df["Type"] = df["Montant operation"].apply(lambda x: "Recette" if x > 0 else "Dépense")
df["Jour"] = df["Date operation"].dt.day
df["Mois"] = df["Date operation"].dt.month
df["Année"] = df["Date operation"].dt.year
df["Mois-Annee"] = df["Date operation"].dt.strftime("%m-%Y")

In [98]:
df.dtypes

id                                  object
Date operation              datetime64[ns]
Categorie operation                 object
Sous Categorie operation            object
Libelle operation                   object
Montant operation                  float64
Date Extraction             datetime64[ns]
Jour                                 int32
Mois                                 int32
Année                                int32
Mois-Annee                          object
is_subscription                       bool
Type                                object
dtype: object

In [99]:
# Filtrer uniquement les dépenses (montant négatif)
data = df
data = data[data["Montant operation"] < 0]
data = data[data["Sous Categorie operation"] != 'Virement interne']
data = data[data["Categorie operation"] != 'Revenus']
df_expenses = data
df_expenses["Montant operation"] = df_expenses["Montant operation"].abs()  # Valeur absolue

In [100]:
# # Regrouper par catégorie et sommer les montants
# category_expenses = df_expenses.groupby("Categorie operation")["Montant operation"].sum().reset_index()
# # Créer le pie chart
# fig1 = px.pie(
#     category_expenses,
#     names="Categorie operation",
#     values="Montant operation",
#     title="Répartition des dépenses par catégorie",
#     hole=0.4,
#     color_discrete_sequence=px.colors.qualitative.Set3 
# )

# fig1.show()


# # Agréger les dépenses par jour
# daily_expenses = df_expenses.groupby("Date operation")["Montant operation"].sum().reset_index()

# # Créer le graphique en ligne
# fig2 = px.line(
#     daily_expenses,
#     x="Date operation",
#     y="Montant operation",
#     title="Évolution des dépenses dans le temps",
#     markers=True,  # Ajoute des points pour chaque jour
#     line_shape="spline",  # Lissage des lignes
#     color_discrete_sequence=["red"]
# )

# fig2.update_layout(
#     xaxis_title="Date",
#     yaxis_title="Montant (€)",
#     xaxis=dict(showgrid=True),
#     yaxis=dict(showgrid=True)
# )

# fig2.show()

Test abonnement avec clusters sur le nom

In [101]:
import pandas as pd
from rapidfuzz import process, fuzz
import re

# Liste des libellés d'exemple
libelles = [
    "PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL - CARTE*5333",
    "PAIEMENT CB ADOBE DU 13/02/25 A SAGGART, DUBL - CARTE*5333",
    "PAIEMENT CB ADOBE DU 13/01/25 A SAGGART, DUBL - CARTE*5333",
    "PAIEMENT CB ADOBE DU 13/12/24 A SAGGART, DUBL - CARTE*5333",
    "PAIEMENT CB ADOBE DU 13/11/24 A SAGGART, DUBL - CARTE*5333",
    "PAIEMENT CB ADOBE DU 13/10/24 A SAGGART, DUBL - CARTE*5333",
    "PAIEMENT CB ADOBE DU 13/09/24 A SAGGART, DUBL - CARTE*5333",
    "PAIEMENT CB ADOBE DU 13/08/24 A SAGGART, DUBL - CARTE*5333",
    "PAIEMENT CB ADOBE (IRLANDE) DU 14/07/24 - CARTE*5333",
    "PAIEMENT CB ADOBE (IRLANDE) DU 13/06/24 - CARTE*5333",
    "PAIEMENT CB ADOBE (IRLANDE) DU 13/05/24 - CARTE*5333",
    "PAIEMENT CB ADOBE (IRLANDE) DU 13/04/24 - CARTE*5333",
    "PAIEMENT CB ADOBE (IRLANDE) DU 13/03/24 - CARTE*5333"
]

# Fonction de nettoyage avancé
def nettoyer_libelle(libelle):
    libelle = libelle.lower()  # Mise en minuscule pour éviter les différences de casse
    libelle = re.sub(r"du \d{1,2}/\d{1,2}/\d{2,4}", "", libelle)  # Supprime les dates
    libelle = re.sub(r"- carte\*\d{4}", "", libelle)  # Supprime la mention carte
    # libelle = re.sub(r"a .+?,", "", libelle)  # Supprime la localisation après "A "
    # libelle = re.sub(r"\(.*?\)", "", libelle)  # Supprime tout ce qui est entre parenthèses (ex: (IRLANDE))
    libelle = re.sub(r"\s+", " ", libelle).strip()  # Supprime les espaces en trop
    return libelle

# Nettoyage des libellés
df = pd.DataFrame({"Libelle brut": libelles})
df["Libelle nettoyé"] = df["Libelle brut"].apply(nettoyer_libelle)

# Regrouper par similarité avec un seuil ajusté
clusters = {}

for libelle in df["Libelle nettoyé"].unique():
    if clusters.keys():
        match, score, _ = process.extractOne(libelle, clusters.keys(), scorer=fuzz.ratio)
        if score > 85:  # Seuil ajusté pour fusionner "PAIEMENT CB ADOBE" et "PAIEMENT CB ADOBE (IRLANDE)"
            clusters[match].append(libelle)
        else:
            clusters[libelle] = [libelle]
    else:
        clusters[libelle] = [libelle]  # Premier élément ajouté directement

# Assigner chaque libellé à son cluster principal
df["Cluster"] = df["Libelle nettoyé"].apply(lambda x: next(k for k, v in clusters.items() if x in v))

df


,Libelle brut,Libelle nettoyé,Cluster
0,"PAIEMENT CB ADOBE DU 13/03/25 A SAGGART, DUBL ...","paiement cb adobe a saggart, dubl","paiement cb adobe a saggart, dubl"
1,"PAIEMENT CB ADOBE DU 13/02/25 A SAGGART, DUBL ...","paiement cb adobe a saggart, dubl","paiement cb adobe a saggart, dubl"
2,"PAIEMENT CB ADOBE DU 13/01/25 A SAGGART, DUBL ...","paiement cb adobe a saggart, dubl","paiement cb adobe a saggart, dubl"
3,"PAIEMENT CB ADOBE DU 13/12/24 A SAGGART, DUBL ...","paiement cb adobe a saggart, dubl","paiement cb adobe a saggart, dubl"
4,"PAIEMENT CB ADOBE DU 13/11/24 A SAGGART, DUBL ...","paiement cb adobe a saggart, dubl","paiement cb adobe a saggart, dubl"
5,"PAIEMENT CB ADOBE DU 13/10/24 A SAGGART, DUBL ...","paiement cb adobe a saggart, dubl","paiement cb adobe a saggart, dubl"
6,"PAIEMENT CB ADOBE DU 13/09/24 A SAGGART, DUBL ...","paiement cb adobe a saggart, dubl","paiement cb adobe a saggart, dubl"
7,"PAIEMENT CB ADOBE DU 13/08/24 A SAGGART, DUBL ...","paiement cb adobe a saggart, dubl","paiement cb adobe a saggart, dubl"
8,PAIEMENT CB ADOBE (IRLANDE) DU 14/07/24 - CART...,paiement cb adobe (irlande),paiement cb adobe (irlande)
9,PAIEMENT CB ADOBE (IRLANDE) DU 13/06/24 - CART...,paiement cb adobe (irlande),paiement cb adobe (irlande)


Test abonnement juste avec reccurence d'achat

In [102]:
import pandas as pd

# # Exemple de données
data = {
    'Date operation': pd.to_datetime([
        '2024-01-15', '2024-01-17', '2024-02-14', '2024-03-16',  # Adobe (exemple récurrent)
        '2024-01-10', '2024-02-10', '2024-03-10',  # Netflix (autre abonnement)
        '2024-01-05', '2024-02-06',  # Un paiement non récurrent sur 3 mois
    ]),
    'Libelle operation': [
        'PAIEMENT CB ADOBE DU 15/01/24',
        'PAIEMENT CB ADOBE DU 15/01/24',
        'PAIEMENT CB ADOBE DU 14/02/24',
        'PAIEMENT CB ADOBE DU 16/03/24',
        'PAIEMENT CB NETFLIX DU 10/01/24',
        'PAIEMENT CB NETFLIX DU 10/02/24',
        'PAIEMENT CB NETFLIX DU 10/03/24',
        'ACHAT UNIQUE DU 05/01/24',
        'ACHAT UNIQUE DU 06/02/24',
    ],
    'Montant operation': [15.99,15.99, 15.99, 15.99, 10.99, 10.99, 10.99, 50.00, 50.00]
}
# data = {
#     'Date operation': pd.to_datetime([
#         '2024-01-15', '2024-02-14', '2024-03-16',  # Adobe (exemple récurrent)
#         '2024-05-10', '2024-04-10', '2024-06-10',  # Netflix (autre abonnement)
#         '2024-01-05', '2024-02-06',  # Un paiement non récurrent sur 3 mois
#     ]),
#     'Libelle operation': [
#         'PAIEMENT CB ADOBE DU 15/01/24',
#         'PAIEMENT CB ADOBE DU 14/02/24',
#         'PAIEMENT CB ADOBE DU 16/03/24',
#         'PAIEMENT CB NETFLIX DU 10/01/24',
#         'PAIEMENT CB NETFLIX DU 10/02/24',
#         'PAIEMENT CB NETFLIX DU 10/03/24',
#         'ACHAT UNIQUE DU 05/01/24',
#         'ACHAT UNIQUE DU 06/02/24',
#     ],
#     'Montant operation': [15.99, 15.99, 15.99, 10.99, 10.99, 10.99, 50.00, 50.00]
# }

df = pd.DataFrame(data)

# Extraction du mois et de l'année pour grouper plus facilement
df['Mois'] = df['Date operation'].dt.to_period('M')

# Comptage du nombre d'occurrences par mois et par montant
monthly_counts = df.groupby(['Montant operation', 'Mois'])['Libelle operation'].count().reset_index()
print(monthly_counts)

# Filtrage des montants qui apparaissent exactement 1 fois par mois
valid_monthly = monthly_counts[monthly_counts['Libelle operation'] == 1]

# Comptage du nombre de mois distincts pour chaque montant
subscription_candidates = valid_monthly.groupby('Montant operation')['Mois'].nunique().reset_index()
print(subscription_candidates)

# Filtrer les montants qui apparaissent sur au moins 3 mois consécutifs
subscription_candidates = subscription_candidates[subscription_candidates['Mois'] >= 3]
print(subscription_candidates)

# Filtrer les transactions originales qui correspondent à ces montants
# df_filtered = df[df['Montant operation'].isin(subscription_candidates['Montant operation'])]
df['is_subscription'] = df['Montant operation'].isin(subscription_candidates['Montant operation'])

df

   Montant operation     Mois  Libelle operation
0              10.99  2024-01                  1
1              10.99  2024-02                  1
2              10.99  2024-03                  1
3              15.99  2024-01                  2
4              15.99  2024-02                  1
5              15.99  2024-03                  1
6              50.00  2024-01                  1
7              50.00  2024-02                  1
   Montant operation  Mois
0              10.99     3
1              15.99     2
2              50.00     2
   Montant operation  Mois
0              10.99     3


,Date operation,Libelle operation,Montant operation,Mois,is_subscription
0,2024-01-15,PAIEMENT CB ADOBE DU 15/01/24,15.99,2024-01,False
1,2024-01-17,PAIEMENT CB ADOBE DU 15/01/24,15.99,2024-01,False
2,2024-02-14,PAIEMENT CB ADOBE DU 14/02/24,15.99,2024-02,False
3,2024-03-16,PAIEMENT CB ADOBE DU 16/03/24,15.99,2024-03,False
4,2024-01-10,PAIEMENT CB NETFLIX DU 10/01/24,10.99,2024-01,True
5,2024-02-10,PAIEMENT CB NETFLIX DU 10/02/24,10.99,2024-02,True
6,2024-03-10,PAIEMENT CB NETFLIX DU 10/03/24,10.99,2024-03,True
7,2024-01-05,ACHAT UNIQUE DU 05/01/24,50.00,2024-01,False
8,2024-02-06,ACHAT UNIQUE DU 06/02/24,50.00,2024-02,False


In [103]:
import pandas as pd

def detect_subscriptions(df):
    """
    Ajoute une colonne 'is_subscription' au dataframe en détectant les paiements récurrents 
    sur au moins 3 mois consécutifs avec exactement le même montant.

    :param df: DataFrame contenant au moins les colonnes 'Date operation' et 'Montant operation'
    :return: DataFrame avec la colonne 'is_subscription' ajoutée
    """
    # S'assurer que la colonne 'Date operation' est bien en datetime
    df['Date operation'] = pd.to_datetime(df['Date operation'])

    # Extraire le mois et l'année pour grouper plus facilement
    df['Mois'] = df['Date operation'].dt.to_period('M')

    # Comptage du nombre d'occurrences par mois et par montant
    monthly_counts = df.groupby(['Montant operation', 'Mois'])['Libelle operation'].count().reset_index()

    # Filtrage des montants qui apparaissent exactement 1 fois par mois
    valid_monthly = monthly_counts[monthly_counts['Libelle operation'] == 1]

    # Comptage du nombre de mois distincts pour chaque montant
    subscription_candidates = valid_monthly.groupby('Montant operation')['Mois'].nunique().reset_index()

    # Identifier les montants correspondant à un abonnement (au moins 3 mois consécutifs)
    subscription_candidates = subscription_candidates[subscription_candidates['Mois'] >= 3]

    # Ajouter la colonne is_subscription au dataframe original
    df['is_subscription'] = df['Montant operation'].isin(subscription_candidates['Montant operation'])

    # Supprimer la colonne temporaire 'Mois'
    df.drop(columns=['Mois'], inplace=True)

    return df


In [107]:
def add_single_month_payment_flag(df):
    """
    Ajoute une colonne 'is_single_month_payment' au DataFrame,
    indiquant si une opération apparaît une seule fois dans le mois.

    Args:
        df (pd.DataFrame): DataFrame contenant une colonne 'Date operation' et 'Libelle operation'.

    Returns:
        pd.DataFrame: DataFrame avec la nouvelle colonne 'is_single_month_payment'.
    """
    # # S'assurer que 'Date operation' est bien de type datetime
    # df = df.copy()  # Pour éviter de modifier l'original
    df["Date operation"] = pd.to_datetime(df["Date operation"])

    # Ajouter la colonne 'Mois-Annee'
    df["Mois-Annee"] = df["Date operation"].dt.strftime("%m-%Y")

    # Compter les occurrences de chaque opération dans un mois donné
    df["operation_count"] = df.groupby(["Mois-Annee", "Libelle operation"])["Libelle operation"].transform("count")

    # Marquer les opérations uniques dans le mois
    df["is_single_month_payment"] = df["operation_count"] == 1

    # Supprimer la colonne temporaire
    df.drop(columns=["operation_count"], inplace=True)

    return df

In [109]:
# Chargement du DataFrame (exemple)
# data = {
#     'Date operation': ['2024-01-15', '2024-02-14', '2024-03-16', '2024-01-10', '2024-02-10', '2024-03-10', '2024-01-05', '2024-02-06'],
#     'Libelle operation': [
#         'PAIEMENT CB ADOBE DU 15/01/24',
#         'PAIEMENT CB ADOBE DU 14/02/24',
#         'PAIEMENT CB ADOBE DU 16/03/24',
#         'PAIEMENT CB NETFLIX DU 10/01/24',
#         'PAIEMENT CB NETFLIX DU 10/02/24',
#         'PAIEMENT CB NETFLIX DU 10/03/24',
#         'ACHAT UNIQUE DU 05/01/24',
#         'ACHAT UNIQUE DU 06/02/24',
#     ],
#     'Montant operation': [15.99, 15.99, 15.99, 10.99, 10.99, 10.99, 50.00, 50.00]
# }
data = {
    'Date operation': pd.to_datetime([
        '2024-01-15', '2024-01-17', '2024-02-14', '2024-03-16',  # Adobe (exemple récurrent)
        '2024-01-10', '2024-02-10', '2024-03-10',  # Netflix (autre abonnement)
        '2024-01-05', '2024-02-06',  # Un paiement non récurrent sur 3 mois
    ]),
    'Libelle operation': [
        'PAIEMENT CB ADOBE DU 15/01/24',
        'PAIEMENT CB ADOBE DU 15/01/24',
        'PAIEMENT CB ADOBE DU 14/02/24',
        'PAIEMENT CB ADOBE DU 16/03/24',
        'PAIEMENT CB NETFLIX DU 10/01/24',
        'PAIEMENT CB NETFLIX DU 10/02/24',
        'PAIEMENT CB NETFLIX DU 10/03/24',
        'ACHAT UNIQUE DU 05/01/24',
        'ACHAT UNIQUE DU 06/02/24',
    ],
    'Montant operation': [15.99,15.99, 15.99, 15.99, 10.99, 10.99, 10.99, 50.00, 50.00]
}

df = pd.DataFrame(data)

# Application de la fonction
# df = detect_subscriptions(df)
df = add_single_month_payment_flag(df)

df


,Date operation,Libelle operation,Montant operation,Mois-Annee,is_single_month_payment
0,2024-01-15,PAIEMENT CB ADOBE DU 15/01/24,15.99,01-2024,False
1,2024-01-17,PAIEMENT CB ADOBE DU 15/01/24,15.99,01-2024,False
2,2024-02-14,PAIEMENT CB ADOBE DU 14/02/24,15.99,02-2024,True
3,2024-03-16,PAIEMENT CB ADOBE DU 16/03/24,15.99,03-2024,True
4,2024-01-10,PAIEMENT CB NETFLIX DU 10/01/24,10.99,01-2024,True
5,2024-02-10,PAIEMENT CB NETFLIX DU 10/02/24,10.99,02-2024,True
6,2024-03-10,PAIEMENT CB NETFLIX DU 10/03/24,10.99,03-2024,True
7,2024-01-05,ACHAT UNIQUE DU 05/01/24,50.00,01-2024,True
8,2024-02-06,ACHAT UNIQUE DU 06/02/24,50.00,02-2024,True
